# ConversationBufferWindowMemory

Hướng dẫn này giới thiệu `ConversationBufferWindowMemory`, giúp duy trì một sliding window các tương tác hội thoại gần đây theo thời gian.

Thay vì sử dụng toàn bộ nội dung hội thoại, `ConversationBufferWindowMemory` chỉ giữ lại **K tương tác gần đây nhất**.

Điều này ngăn buffer memory phát triển quá lớn.


In [1]:
# Setup environment
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.globals import set_llm_cache
from langchain_community.cache import SQLiteCache

load_dotenv(override=True, dotenv_path="../.env")

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.0)
prompt = PromptTemplate.from_template("Liệt kê 5 món ăn ngon ở {place}")
chain = prompt | llm | StrOutputParser()

# Create cache directory
if not os.path.exists("cache"):
    os.makedirs("cache")

# Set SQLiteCache
set_llm_cache(SQLiteCache(database_path="cache/llm_cache.db"))

answer = chain.invoke("Hà Nội")
print(answer)

1. Phở Hà Nội: Một trong những món ăn nổi tiếng và phổ biến nhất của Hà Nội, phở Hà Nội có hương vị đặc trưng, ngon và hấp dẫn.

2. Bún chả: Một món ăn truyền thống của Hà Nội, bún chả gồm bún, thịt nướng và nước mắm chua ngọt, tạo nên hương vị đặc trưng và hấp dẫn.

3. Bánh mỳ pate: Một món ăn sáng phổ biến ở Hà Nội, bánh mỳ pate có vị ngon, thơm và béo của pate kết hợp với vị giòn của bánh mỳ.

4. Nem chua rán: Một món ăn vặt phổ biến của người dân Hà Nội, nem chua rán có vị chua, cay, ngọt và giòn rất hấp dẫn.

5. Bún ốc: Một món ăn đặc trưng của Hà Nội, bún ốc có hương vị đậm đà, thơm ngon và hấp dẫn, được nhiều người yêu thích.


## Online Bank Account Opening Conversation Example

Ví dụ này minh họa việc sử dụng `ConversationBufferWindowMemory` để mô phỏng một cuộc hội thoại trợ lý ngân hàng ảo. Luồng này bao gồm quy trình mở tài khoản ngân hàng trực tuyến thông thường, từ lời chào ban đầu đến xác nhận tạo tài khoản, chỉ giữ lại các tương tác gần đây nhất trong memory.


In [2]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2, return_messages=True)

memory.save_context(
    inputs={
        "human": "Hello, I'd like to open a bank account online. How do I get started?"
    },
    outputs={
        "ai": "Hello! I'm glad you want to open an account. First, could you please prepare your ID for verification?"
    },
)
memory.save_context(
    inputs={"human": "Yes, I have my ID ready. What should I do next?"},
    outputs={
        "ai": "Thank you. Please take clear photos of both sides of your ID and upload them. Then we'll proceed with the identity verification process."
    },
)
memory.save_context(
    inputs={
        "human": "I've uploaded the photos. How does the identity verification work?"
    },
    outputs={
        "ai": "I've confirmed your uploaded photos. Now, please proceed with phone verification. You'll receive a verification code via text message - please enter that code."
    },
)
memory.save_context(
    inputs={
        "human": "I've entered the verification code. How do I proceed with opening the account?"
    },
    outputs={
        "ai": "Your identity has been verified. Now, please select the type of account you want and enter the required information. You can choose the type of deposit, currency, and other options."
    },
)
memory.save_context(
    inputs={"human": "I've entered all the information. What's the next step?"},
    outputs={
        "ai": "I've confirmed your information. The account opening process is almost complete. Finally, please agree to the terms of service and confirm the account opening."
    },
)
memory.save_context(
    inputs={"human": "I've completed all the steps. Has the account been opened?"},
    outputs={
        "ai": "Yes, your account has been successfully opened. Your account number and related information have been sent to your registered email. Please let us know if you need any further assistance. Thank you!"
    },
)

/tmp/ipykernel_732502/2111032156.py:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(k=2, return_messages=True)


## Retrieving Conversation History

Let's examine the stored conversation history using the `load_memory_variables()` method to verify our window-based memory retention.

In [3]:
# Check the conversation history
memory.load_memory_variables({})["history"]

[HumanMessage(content="I've entered all the information. What's the next step?", additional_kwargs={}, response_metadata={}),
 AIMessage(content="I've confirmed your information. The account opening process is almost complete. Finally, please agree to the terms of service and confirm the account opening.", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="I've completed all the steps. Has the account been opened?", additional_kwargs={}, response_metadata={}),
 AIMessage(content='Yes, your account has been successfully opened. Your account number and related information have been sent to your registered email. Please let us know if you need any further assistance. Thank you!', additional_kwargs={}, response_metadata={})]